# CIGRE MV Powerflow Solver Test

## Run simulation

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_With_LoadFlow_Results/Rootnet_FULL_NE_06J16h'
filename = 'CIGRE-MV'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import villas.dataprocessing.validationtools as validationtools
import dpsimpy

In [ ]:
name = 'CIGRE_MV'
reader = dpsimpy.CIMReader(name)
system = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
system

In [ ]:
sim = dpsimpy.Simulation(name)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)

logger = dpsimpy.Logger(name)
for node in system.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node);
sim.add_logger(logger)

In [ ]:
sim.run()

## Read DPsim results

In [ ]:
path = 'logs/'
logName = 'CIGRE_MV'
dpsim_result_file = path + logName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

# Fix for dpsim naming - TODO: unify dpsim notation in log file and update villas-dataprocessing accordingly
for ts,values in ts_dpsim.items():
    values.name = values.name[:-2]

## Read NEPLAN results as reference

In [ ]:
if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/Neplan/ReferenceGrids/CIGRE_MV.rlf'
neplan_result_file = 'reference-results/CIGRE_MV.rlf'
urllib.request.urlretrieve(url, neplan_result_file) 
print(neplan_result_file)

ts_NEPLAN = read_timeseries_NEPLAN_loadflow(neplan_result_file)
#print([ts_NEPLAN[i].name for i in range(len(ts_NEPLAN))])

## Comparison and assertion of results

In [ ]:
net_name='CIGRE_MV_NoTap'
threshold=0.5

ts_NEPLAN_standardized=validationtools.convert_neplan_to_standard_timeseries(ts_NEPLAN)
#print([ts_NEPLAN_standardized[i].name for i in range(len(ts_NEPLAN_standardized))])
ts_dpsim_standardized=validationtools.convert_dpsim_to_standard_timeseries(ts_dpsim)
#print([ts_dpsim_standardized[i].name for i in range(len(ts_dpsim_standardized))])
res_err=validationtools.compare_timeseries(ts_NEPLAN_standardized,ts_dpsim_standardized)
validationtools.assert_modelica_results(net_name,res_err,threshold)